---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
df.describe()

count                               500
unique                              500
top       e12/2007 dx of endocarditis\n
freq                                  1
dtype: object

In [4]:
# 4.1 Cardinality-Grammar
ZERO_OR_MORE = '*'
ONE_OR_MORE = "+"
ZERO_OR_ONE = '?'
EXACTLY_TWO = "{2}"
ONE_OR_TWO = "{1,2}"
EXACTLY_ONE = '{1}'

In [5]:
# 4.2 Groups and Classes
GROUP = r"({})"
NAMED = r"(?P<{}>{})"
CLASS = "[{}]"
NEGATIVE_LOOKAHEAD = "(?!{})"
NEGATIVE_LOOKBEHIND = "(?<!{})"
POSITIVE_LOOKAHEAD = "(?={})"
POSITIVE_LOOKBEHIND = "(?<={})"
ESCAPE = "\{}"

In [6]:
#4.3 Numbers
DIGIT = r"\d"
ONE_DIGIT  = DIGIT + EXACTLY_ONE
ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO
NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
TWO_DIGITS = DIGIT + EXACTLY_TWO
THREE_DIGITS = DIGIT + "{3}"
EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
FOUR_DIGITS = DIGIT + r"{4}" + NON_DIGIT

In [7]:
# 4.4 String Literals
SLASH = r"/"
OR = r'|'
LOWER_CASE = "a-z"
SPACE = "\s"
DOT = "."
DASH = "-"
COMMA = ","
PUNCTUATION = CLASS.format(DOT + COMMA + DASH)
EMPTY_STRING = ""

In [8]:
# 4.5 Dates
# These are parts to build up the date-expressions.
MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
                + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
MONTHS = [month + MONTH_SUFFIX for month in MONTH_PREFIXES]
MONTHS = GROUP.format(OR.join(MONTHS))
DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
YEAR = FOUR_DIGITS

In [9]:
# This is for dates like Mar 21st, 2009, those with suffixes on the days.
CONTRACTED = (ONE_OR_TWO_DIGITS
              + LOWER_CASE
              + EXACTLY_TWO
              )
CONTRACTION = NAMED.format("contraction",MONTHS+ CONTRACTED+ DAY_SUFFIX+ YEAR)

In [10]:
# This is for dates that have no days in them, like May 2009.
NO_DAY_BEHIND = NEGATIVE_LOOKBEHIND.format(DIGIT + SPACE)
NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)

In [11]:
WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)

In [12]:
BACKWARDS = NAMED.format("backwards", ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)

In [13]:
slashed = SLASH.join([ONE_OR_TWO_DIGITS,
                      ONE_OR_TWO_DIGITS,
                      EXACTLY_TWO_DIGITS])
dashed = DASH.join([ONE_OR_TWO_DIGITS,
                    ONE_OR_TWO_DIGITS,
                    EXACTLY_TWO_DIGITS])
TWENTIETH_CENTURY = NAMED.format("twentieth",
                                 OR.join([slashed, dashed]))

In [14]:
NUMERIC = NAMED.format("numeric",
                       SLASH.join([ONE_OR_TWO_DIGITS,
                                   ONE_OR_TWO_DIGITS,
                                   FOUR_DIGITS]))

In [15]:
NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHIND.format(SLASH)
NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHIND.format(CLASS.format(SLASH + DIGIT))
NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
              + ONE_DIGIT
              + SLASH
              + FOUR_DIGITS)
NO_TWO_DAYS = (NO_PRECEDING_SLASH
               + TWO_DIGITS
               + SLASH
               + FOUR_DIGITS)
NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                              NO_ONE_DAY
                              + OR
                              + NO_TWO_DAYS
                              )

In [16]:
CENTURY = GROUP.format('19' + OR + "20") + TWO_DIGITS
DIGIT_SLASH = DIGIT + SLASH
DIGIT_DASH = DIGIT + DASH
DIGIT_SPACE = DIGIT + SPACE
LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
COMMA_SPACE = COMMA + SPACE
YEAR_PREFIX = NEGATIVE_LOOKBEHIND.format(OR.join([
    DIGIT_SLASH,
    DIGIT_DASH,
    DIGIT_SPACE,
    LETTER_SPACE,
    COMMA_SPACE,
]))

YEAR_ONLY = NAMED.format("year_only",
                         YEAR_PREFIX + CENTURY
)

In [17]:
IN_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
SINCE_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format("Ss") + 'ince' + SPACE) + CENTURY
AGE = POSITIVE_LOOKBEHIND.format("Age" + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
AGE_COMMA = POSITIVE_LOOKBEHIND.format("Age" + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
OTHERS = ['delivery', "quit", "attempt", "nephrectomy", THREE_DIGITS]
OTHERS = [POSITIVE_LOOKBEHIND.format(label + SPACE) + CENTURY for label in OTHERS]
OTHERS = OR.join(OTHERS)
LEFTOVERS_PREFIX = OR.join([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
LEFTOVERS = NAMED.format("leftovers", LEFTOVERS_PREFIX)

In [18]:
DATE = NAMED.format("date", OR.join([NUMERIC,
                                     TWENTIETH_CENTURY,
                                     WORDS,
                                     BACKWARDS,
                                     CONTRACTION,
                                     NO_DAY,
                                     NO_DAY_NUMERIC,
                                     YEAR_ONLY,
                                     LEFTOVERS]))

In [19]:
def twentieth_century(date):
    """adds a 19 to the year

    Args:
     date (re.Regex): Extracted date
    """
    month, day, year = date.group(1).split(SLASH)
    year = "19{}".format(year)
    return SLASH.join([month, day, year])

In [20]:
def take_two(line):
    match = re.search(TWENTIETH_CENTURY, line)
    if match:
        return twentieth_century(match)
    return line

In [21]:
def extract_and_count(expression, data, name):
    """extract all matches and report the count

    Args:
     expression (str): regular expression to match
     data (pandas.Series): data with dates to extratc
     name (str): name of the group for the expression

    Returns:
     tuple (pandas.Series, int): extracted dates, count
    """
    extracted = data.str.extractall(expression)[name]
    count = len(extracted)
    print("'{}' matched {} rows".format(name, count))
    return extracted, count

In [23]:
numeric, numeric_count = extract_and_count(NUMERIC, df, 'numeric')
# 'numeric' matched 25 rows
twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, df, 'twentieth')
# 'twentieth' matched 100 rows
words, words_count = extract_and_count(WORDS, df, 'words')
# 'words' matched 34 rows
backwards, backwards_count = extract_and_count(BACKWARDS, df, 'backwards')
# 'backwards' matched 69 rows
contraction_data, contraction = extract_and_count(CONTRACTION, df, 'contraction')
# 'contraction' matched 0 rows
no_day, no_day_count = extract_and_count(NO_DAY, df, 'no_day')
# 'no_day' matched 115 rows
no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, df,
                                                         "no_day_numeric")
# 'no_day_numeric' matched 112 rows
year_only, year_only_count = extract_and_count(YEAR_ONLY, df, "year_only")
# 'year_only' matched 15 rows
leftovers, leftovers_count = extract_and_count(LEFTOVERS, df, "leftovers")
# 'leftovers' matched 30 rows

'numeric' matched 25 rows
'twentieth' matched 100 rows
'words' matched 34 rows
'backwards' matched 69 rows
'contraction' matched 0 rows
'no_day' matched 115 rows
'no_day_numeric' matched 112 rows
'year_only' matched 15 rows
'leftovers' matched 30 rows


In [25]:
found = df.str.extractall(DATE)
total_found = len(found.date)

print("Total Found: {}".format(total_found))
print("Remaining: {}".format(len(df) - total_found))
print("Discrepancy: {}".format(total_found - (numeric_count
                                              + twentieth_count
                                              + words_count
                                              + backwards_count
                                              + contraction
                                              + no_day_count
                                              + no_day_numeric_count
                                              + year_only_count
                                              + leftovers_count)))

Total Found: 500
Remaining: 0
Discrepancy: 0


In [26]:
missing = [label for label in df.index if label not in found.index.levels[0]]
try:
    print(missing[0], df.loc[missing[0]])
except IndexError:
    print("all rows matched")

all rows matched


In [27]:
def clean(source, expression, replacement, sample=5):
    """applies the replacement to the source

    as a side-effect shows sample rows before and after

    Args:
     source (pandas.Series): source of the strings
     expression (str): regular expression to match what to replace
     replacement: function or expression to replace the matching expression
     sample (int): number of randomly chosen examples to show

    Returns:
     pandas.Series: the source with the replacement applied to it
    """
    print("Random Sample Before:")
    print(source.sample(sample))
    cleaned = source.str.replace(expression, replacement)
    print("\nRandom Sample After:")
    print(cleaned.sample(sample))
    print("\nCount of cleaned: {}".format(len(cleaned)))
    assert len(source) == len(cleaned)
    return cleaned

In [28]:
def clean_punctuation(source, sample=5):
    """removes punctuation

    Args:
     source (pandas.Series): data to clean
     sample (int): size of sample to show

    Returns:
     pandas.Series: source with punctuation removed
    """
    print("Cleaning Punctuation")
    if any(source.str.contains(PUNCTUATION)):
        source = clean(source, PUNCTUATION, EMPTY_STRING)
    return source

In [29]:
LONG_TO_SHORT = dict(January="Jan",
                     February="Feb",
                     March="Mar",
                     April="Apr",
                     May="May",
                     June="Jun",
                     July="Jul",
                     August="Aug",
                     September="Sep",
                     October="Oct",
                     November="Nov",
                     December="Dec")

# it turns out there are spelling errors in the data so this has to be fuzzy
LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                    + CLASS.format(LOWER_CASE)
                                    + ZERO_OR_MORE
                                    for month in LONG_TO_SHORT.values()])

def long_month_to_short(match):
    """convert long month to short

    Args:
     match (re.Match): object matching a long month

    Returns:
     str: shortened version of the month
    """
    return match.group(match.lastindex)

In [30]:
def convert_long_months_to_short(source, sample=5):
    """convert long month names to short

    Args:
     source (pandas.Series): data with months
     sample (int): size of sample to show

    Returns:
     pandas.Series: data with short months
    """
    return clean(source,
                 LONG_TO_SHORT_EXPRESSION,
                 long_month_to_short)

In [31]:
def add_month_date(match):
    """adds 01/01 to years

    Args:
     match (re.Match): object that only matched a 4-digit year

    Returns:
     str: 01/01/YYYY
    """
    return "01/01/" + match.group()

In [32]:
def add_january_one(source):
    """adds /01/01/ to year-only dates

    Args:
     source (pandas.Series): data with the dates

    Returns:
     pandas.Series: years in source with /01/01/ added
    """
    return clean(source, YEAR_ONLY, add_month_date)

In [33]:
two_digit_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH)

def two_digits(match):
    """add a leading zero if needed

    Args:
     match (re.Match): match with one or two digits

    Returns:
     str: the matched string with leading zero if needed
    """
    # for some reason the string-formatting raises an error if it's a string
    # so cast it to an int
    return "{:02}".format(int(match.group()))

In [34]:
def clean_two_digits(source, sample=5):
    """makes sure source has two-digits

    Args:
     source (pandas.Series): data with digit followed by slash
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with digits coerced to two digits
    """
    return clean(source, two_digit_expression, two_digits, sample)

In [35]:
def clean_two_digits_isolated(source, sample=5):
    """cleans two digits that are standalone

    Args:
     source (pandas.Series): source of the data
     sample (int): number of samples to show

    Returns:
     pandas.Series: converted data
    """
    return clean(source, ONE_OR_TWO_DIGITS, two_digits, sample)

In [36]:
digits = ("{:02}".format(month) for month in range(1, 13))
MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))
def month_to_digits(match):
    """converts short month to digits

    Args:
     match (re.Match): object with short-month

    Returns:
     str: month as two-digit number (e.g. Jan -> 01)
    """
    return MONTH_TO_DIGITS[match.group()]

In [37]:
def convert_short_month_to_digits(source, sample=5):
    """converts three-letter months to two-digits

    Args:
     source (pandas.Series): data with three-letter months
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with short-months coverted to digits
    """
    return clean(source,
                 SHORT_MONTHS_EXPRESSION,
                 month_to_digits,
                 sample)

In [38]:
def clean_months(source, sample=5):

    """clean up months (which start as words)

    Args:
     source (pandas.Series): source of the months
     sample (int): number of random samples to show
    """
    cleaned = clean_punctuation(source)

    print("Converting long months to short")
    cleaned = clean(cleaned,
                    LONG_TO_SHORT_EXPRESSION,
                    long_month_to_short, sample)

    print("Converting short months to digits")
    cleaned = clean(cleaned,
                    SHORT_MONTHS_EXPRESSION,
                    month_to_digits, sample)
    return cleaned

In [39]:
def frame_to_series(frame, index_source, samples=5):
    """re-combines data-frame into a series

    Args:
     frame (pandas.DataFrame): frame with month, day, year columns
     index_source (pandas.series): source to copy index from
     samples (index): number of random entries to print when done

    Returns:
     pandas.Series: series with dates as month/day/year
    """
    combined = frame.month + SLASH + frame.day + SLASH + frame.year
    combined.index = index_source.index
    print(combined.sample(samples))
    return combined

In [40]:
year_only_cleaned = add_january_one(year_only)

Random Sample Before:
     match
472  0        2010
462  0        1988
480  0        2013
493  0        1978
483  0        1995
Name: year_only, dtype: object

Random Sample After:
     match
472  0        01/01/2010
483  0        01/01/1995
493  0        01/01/1978
497  0        01/01/2008
481  0        01/01/1974
Name: year_only, dtype: object

Count of cleaned: 15


In [41]:
leftovers_cleaned = add_january_one(leftovers)

Random Sample Before:
     match
457  0        2001
491  0        2009
464  0        2016
471  0        1999
463  0        2014
Name: leftovers, dtype: object

Random Sample After:
     match
490  0        01/01/2007
471  0        01/01/1999
464  0        01/01/2016
463  0        01/01/2014
474  0        01/01/1972
Name: leftovers, dtype: object

Count of cleaned: 30


In [43]:
cleaned = pd.concat([year_only_cleaned, leftovers_cleaned])
print(len(cleaned))

45


In [44]:
no_day_numeric_cleaned = clean_two_digits(no_day_numeric)

Random Sample Before:
     match
418  0         8/1975
399  0         1/1992
374  0        11/2000
442  0         9/1992
356  0         9/2003
Name: no_day_numeric, dtype: object

Random Sample After:
     match
424  0        04/1979
355  0        03/1981
377  0        06/2001
354  0        03/1993
415  0        02/1973
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [45]:
no_day_numeric_cleaned = clean(no_day_numeric_cleaned,
                               SLASH,
                               lambda m: "/01/")

Random Sample Before:
     match
363  0        12/1975
445  0        01/2008
364  0        11/2010
384  0        05/1999
419  0        05/1977
Name: no_day_numeric, dtype: object

Random Sample After:
     match
376  0        02/01/1999
361  0        10/01/1980
383  0        12/01/2012
381  0        01/01/2014
423  0        12/01/1986
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [47]:
original = len(cleaned)
cleaned = pd.concat([cleaned, no_day_numeric_cleaned])
assert len(cleaned) == no_day_numeric_count + original

In [48]:
print(len(cleaned))

157


In [49]:
no_day_cleaned = clean_months(no_day)


Cleaning Punctuation
Random Sample Before:
     match
233  0           July, 1990
314  0         January 2007
247  0             May 1983
269  0            July 1992
302  0        November 2004
Name: no_day, dtype: object

Random Sample After:
     match
342  0            March 1976
279  0              Sep 2013
275  0        September 1984
338  0              Apr 1998
252  0              Jan 2007
Name: no_day, dtype: object

Count of cleaned: 115
Converting long months to short
Random Sample Before:
     match
260  0        February 2000
322  0         October 1991
291  0             Jan 2004
250  0             May 2005
257  0             Sep 2015
Name: no_day, dtype: object

Random Sample After:
     match
292  0        Nov 1995
324  0        Oct 1996
232  0        Jul 1977
326  0        Oct 1995
297  0        Jan 2009
Name: no_day, dtype: object

Count of cleaned: 115
Converting short months to digits
Random Sample Before:
     match
244  0        Jan 2013
229  0        Jun 2011
283 

In [50]:
no_day_cleaned = clean(no_day_cleaned,
                       SPACE + ONE_OR_MORE,
                       lambda match: "/01/")

Random Sample Before:
     match
333  0        11 1997
306  0        05 2004
237  0        02 1976
245  0        11 1990
261  0        10 1986
Name: no_day, dtype: object

Random Sample After:
     match
335  0        02/01/1973
315  0        06/01/1976
340  0        05/01/1980
313  0        12/01/1978
324  0        10/01/1996
Name: no_day, dtype: object

Count of cleaned: 115


In [52]:
original = len(cleaned)
cleaned = pd.concat([cleaned, no_day_cleaned])
print(len(cleaned))

272


In [53]:
assert len(cleaned) == no_day_count + original

In [55]:
frame = pd.DataFrame(backwards.str.split().tolist(),
                         columns="day month year".split())
frame.head()

,day,month,year
0,24,Jan,2001
1,10,Sep,2004
2,26,May,1982
3,28,June,2002
4,06,May,1972


In [56]:
frame.day = clean_two_digits(frame.day)


Random Sample Before:
29    06
39    21
1     10
28    13
66    30
Name: day, dtype: object

Random Sample After:
38    29
12    10
45    24
40    18
8     28
Name: day, dtype: object

Count of cleaned: 69


In [57]:
frame.month = clean_months(frame.month)

Cleaning Punctuation
Converting long months to short
Random Sample Before:
39     Oct
3     June
10     Oct
43     Oct
34     May
Name: month, dtype: object

Random Sample After:
40    Aug
5     Oct
28    Jan
12    Feb
56    Aug
Name: month, dtype: object

Count of cleaned: 69
Converting short months to digits
Random Sample Before:
11    Feb
56    Aug
65    Aug
32    Feb
43    Oct
Name: month, dtype: object

Random Sample After:
46    10
60    08
68    01
32    02
31    05
Name: month, dtype: object

Count of cleaned: 69


In [58]:
backwards_cleaned = frame_to_series(frame, backwards)

     match
155  0        10/10/1974
152  0        09/28/2015
128  0        06/28/2002
146  0        11/11/2004
178  0        06/15/1985
dtype: object


In [60]:
original = len(cleaned)
cleaned = pd.concat([cleaned, backwards_cleaned])
assert len(cleaned) == original + backwards_count

In [61]:
print(len(cleaned))

341


In [63]:
frame = pd.DataFrame(words.str.split().tolist(), columns="month day year".split())
print(frame.head())

      month  day  year
0     April  11,  1990
1       May  30,  2001
2       Feb  18,  1994
3  February  18,  1981
4  October.  11,  2013


In [64]:
frame.month = clean_months(frame.month)

Cleaning Punctuation
Random Sample Before:
19        July
13      August
7     December
20        Sep.
17       April
Name: month, dtype: object

Random Sample After:
28        May
25        Dec
19       July
10        Mar
32    January
Name: month, dtype: object

Count of cleaned: 34
Converting long months to short
Random Sample Before:
29    October
11        Jan
5         Jan
26       June
25        Dec
Name: month, dtype: object

Random Sample After:
28    May
11    Jan
26    Jun
4     Oct
15    Jul
Name: month, dtype: object

Count of cleaned: 34
Converting short months to digits
Random Sample Before:
14    Sep
13    Aug
0     Apr
20    Sep
5     Jan
Name: month, dtype: object

Random Sample After:
14    09
18    07
21    08
11    01
5     01
Name: month, dtype: object

Count of cleaned: 34


In [65]:
frame.day = clean_punctuation(frame.day)

Cleaning Punctuation
Random Sample Before:
26    25,
28    15,
13     12
9     06,
21    14,
Name: day, dtype: object

Random Sample After:
26    25
14    01
6     26
30    25
8     15
Name: day, dtype: object

Count of cleaned: 34


In [67]:
frame.head()

,month,day,year
0,04,11,1990
1,05,30,2001
2,02,18,1994
3,02,18,1981
4,10,11,2013


In [68]:
words_cleaned = frame_to_series(frame, words)


     match
214  0        09/10/1974
218  0        05/14/1989
209  0        07/25/1983
211  0        04/17/1992
197  0        02/18/1981
dtype: object


In [70]:
original = len(cleaned)
cleaned = pd.concat([cleaned, words_cleaned])
assert len(cleaned) == original + words_count
print(len(cleaned))

375


In [71]:
print(twentieth.iloc[21])
twentieth_cleaned = twentieth.str.replace(DASH, SLASH)
print(cleaned.iloc[21])

4-13-82
01/01/1991


In [73]:
frame = pd.DataFrame(twentieth_cleaned.str.split(SLASH).tolist(),
                         columns=["month", "day", "year"])
print(frame.head())

  month day year
0    03  25   93
1     6  18   85
2     7   8   71
3     9  27   75
4     2   6   96


In [74]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
33    7
49    8
42    5
73    4
23    9
Name: month, dtype: object

Random Sample After:
47    12
78    11
55    01
14    04
30    02
Name: month, dtype: object

Count of cleaned: 100


In [75]:
frame.day = clean_two_digits_isolated(frame.day)

Random Sample Before:
10    11
56    04
39    17
76    31
98    04
Name: day, dtype: object

Random Sample After:
74    20
32    14
40    28
50    22
99    27
Name: day, dtype: object

Count of cleaned: 100


In [76]:
frame.head()

,month,day,year
0,03,25,93
1,06,18,85
2,07,08,71
3,09,27,75
4,02,06,96


In [77]:
frame.year = clean(frame.year, TWO_DIGITS, lambda match: "19" + match.group())

Random Sample Before:
44    71
32    81
68    93
84    83
47    73
Name: year, dtype: object

Random Sample After:
79    1992
66    1978
57    1989
96    1996
81    1982
Name: year, dtype: object

Count of cleaned: 100


In [78]:
twentieth_cleaned = frame_to_series(frame, twentieth)

     match
50   0        11/22/1975
47   0        06/12/1994
89   0        08/26/1989
56   0        03/14/1995
124  0        02/27/1996
dtype: object


In [79]:
original = len(cleaned)
cleaned = pd.concat([cleaned, twentieth_cleaned])

In [80]:
assert len(cleaned) == original + twentieth_count

In [81]:
print(numeric.head())

    match
14  0         5/24/1990
15  0         1/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
Name: numeric, dtype: object


In [82]:
has_dashes = numeric.str.contains(DASH)
print(numeric[has_dashes])

Series([], Name: numeric, dtype: object)


In [84]:
frame = pd.DataFrame(numeric.str.split(SLASH).tolist(),
                         columns="month day year".split())
print(frame.head())

  month day  year
0     5  24  1990
1     1  25  2011
2    10  13  1976
3    07  25  1984
4    03  31  1985


In [85]:
frame.month = clean_two_digits_isolated(frame.month)

Random Sample Before:
6      7
8      2
7      4
12    07
15     7
Name: month, dtype: object

Random Sample After:
19    12
8     02
22    02
23    07
11    08
Name: month, dtype: object

Count of cleaned: 25


In [86]:
frame.day = clean_two_digits_isolated(frame.day)

Random Sample Before:
15    07
6     27
5     12
24    27
20    11
Name: day, dtype: object

Random Sample After:
0     24
22    11
14    05
19    08
1     25
Name: day, dtype: object

Count of cleaned: 25


In [87]:
numeric_cleaned = frame_to_series(frame, numeric)

    match
38  0        07/27/1986
82  0        07/07/1974
64  0        07/29/1994
59  0        08/20/1982
17  0        10/13/1976
dtype: object


In [89]:
original = len(cleaned)
cleaned = pd.concat([cleaned, numeric_cleaned])
assert len(cleaned) == original + numeric_count
print(len(cleaned))

500


In [91]:
cleaned = pd.concat([numeric_cleaned,
                         twentieth_cleaned,
                         words_cleaned,
                         backwards_cleaned,
                         no_day_cleaned,
                         no_day_numeric_cleaned,
                         year_only_cleaned,
                         leftovers_cleaned,
])
print(len(cleaned))
print(cleaned.head())
assert len(cleaned) == len(df)

500
    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object


In [92]:
print(cleaned.head())
datetimes = pd.to_datetime(cleaned, format="%m/%d/%Y")
print(datetimes.head())

    match
14  0        05/24/1990
15  0        01/25/2011
17  0        10/13/1976
24  0        07/25/1984
30  0        03/31/1985
dtype: object
    match
14  0       1990-05-24
15  0       2011-01-25
17  0       1976-10-13
24  0       1984-07-25
30  0       1985-03-31
dtype: datetime64[ns]


In [93]:
sorted_dates = datetimes.sort_values()
print(sorted_dates.head())

    match
9   0       1971-04-10
84  0       1971-05-18
2   0       1971-07-08
53  0       1971-07-11
28  0       1971-09-12
dtype: datetime64[ns]


In [94]:
print(sorted_dates.tail())

     match
231  0       2016-05-01
141  0       2016-05-30
186  0       2016-10-13
161  0       2016-10-19
413  0       2016-11-01
dtype: datetime64[ns]


In [95]:
answer = pd.Series(sorted_dates.index.labels[0])
print(answer.head())

0     9
1    84
2     2
3    53
4    28
dtype: int16


In [96]:
def date_sorter():
    
    # Your code here
    
    return answer

In [97]:
#date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    480
478    286
479    431
480    279
481    198
482    463
483    381
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int16